<a href="https://colab.research.google.com/github/L120Git/NLP_Google_Restaurants/blob/main/2_NLP_preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##0.Librerías necesarias

In [1]:
#!pip install rarfile
#!pip install unzip
!pip install stop-words
!pip install num2words


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=324bd14de7559170defca22a522c83241be5999a897c183123af2351d82e660c
  Stored in directory: /root/.cache/pip/wheels/da/d8/66/395317506a23a9d1d7de433ad6a7d9e6e16aab48cf028a0f60
Successfully built stop-words
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=7e11b78aed028c972c6d43936a7de4e6bf2f56625734db9a05b4ecefa7f93098
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [15]:
#Librerias necesarias
from google.colab import drive
import numpy as np
import pandas as pd
from google.colab import drive
import random
#import rarfile # para descomprimir archivos
import json
import csv 
#para reducir la dimensión
from sklearn.manifold import TSNE
#para representar
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#preprocesado
import unicodedata
import re
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
from nltk.corpus import stopwords
from stop_words import get_stop_words
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
nltk.download('omw-1.4')
import string
from num2words import num2words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# Montamos GDrive para almacenar los datos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##1. Carga de datos

In [4]:
#cargamos los datos necesarios desde el gDrive
path = '/content/drive/MyDrive/NLP/'

train = pd.read_csv(f'{path}train.csv')
test= pd.read_csv(f'{path}test.csv')

In [5]:
train

,review_text,reviews_sentiment,words,sentiment_label
0,When I want a great slice of New York Pizza I ...,5,"['When', 'I', 'want', 'great', 'slice', 'of', ...",0
1,"Way too much breading on the shrimp, also real...",2,"['Way', 'too', 'much', 'breading', 'on', 'the'...",1
2,It was the first time I had eaten Peruvian foo...,5,"['It', 'was', 'the', 'first', 'time', 'I', 'ha...",0
3,IHop has better food the further you go down s...,3,"['IHop', 'better', 'food', 'the', 'further', '...",1
4,"Bowling, giant jenga, corn hole, ping pong and...",4,"['Bowling', ',', 'giant', 'jenga', ',', 'corn'...",1
...,...,...,...,...
4760,Heres our opinion.... . Frosted Hawaiian- tast...,3,"['Heres', 'our', 'opinion', '....', '.', 'Fros...",1
4761,We had a chicken sandwich and double berger co...,4,"['We', 'had', 'chicken', 'sandwich', 'and', 'd...",1
4762,Came here to try something new for breakfast a...,5,"['Came', 'here', 'to', 'try', 'something', 'ne...",0
4763,My daughter had the blackened fish sandwich. M...,3,"['My', 'daughter', 'had', 'the', 'blackened', ...",1


In [6]:
test

,review_text,reviews_sentiment,words,sentiment_label
0,My wife and I really enjoy the pasta and fresh...,3,"['My', 'wife', 'and', 'I', 'really', 'enjoy', ...",1
1,"Had the pork cracklins, boudin balls and sweet...",5,"['Had', 'the', 'pork', 'cracklins', ',', 'boud...",0
2,"Salad, appetizer, and main course all came out...",2,"['Salad', ',', 'appetizer', ',', 'and', 'main'...",1
3,"The burgers are great, the fries are good but ...",4,"['The', 'burgers', 'are', 'great', ',', 'the',...",1
4,always so bomb this place never disappoints so...,5,"['always', 'so', 'bomb', 'this', 'place', 'nev...",0
...,...,...,...,...
1584,"My burger though was average, something I coul...",3,"['My', 'burger', 'though', 'was', 'average', '...",1
1585,Good flavors but I personally prefer thicker n...,4,"['Good', 'flavors', 'but', 'I', 'personally', ...",1
1586,Worth a one time stop only and only if you're ...,4,"['Worth', 'one', 'time', 'stop', 'only', 'and'...",1
1587,Wasn't fond of the root beer doughnuts (too sw...,3,"['Was', ""n't"", 'fond', 'of', 'the', 'root', 'b...",1


## 2. Preprocesado
#### Se realizan los siguientes pasos:

> 1.   Normalizar
2.   Eliminar símbolos y caracteres especiales
3.   Eliminar signos de puntuación
4.   Lematizar
5.   Eliminar stopwords




In [7]:
#creamos una función que nos haga toda la limpieza

stop_words = set(stopwords.words("english")) #cargamos las stop.words
lemmatizer = WordNetLemmatizer() #cargamos el lematizador

def clean_review(review):
    # Convertir a minúsculas
    review = review.lower()
    
    # Eliminar acentos, etc
    text = unicodedata.normalize('NFKD', review).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Eliminar los signos de puntuación y caracteres especiales de cada palabra
    token = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenizar las palabras
    words = word_tokenize(token)

    # Lematizar las palabras
    words = [lemmatizer.lemmatize(w) for w in words]

    # Eliminar las stopwords
    words = [w for w in words if not w in stop_words]

    # Unir las palabras en una sola cadena
    clean_review = ' '.join(words)

    return clean_review


####2.1 Preprocesamos el conjunto de train

In [8]:
# Aplicar la función a la columna 'review'
train['review_clean'] = train['review_text'].apply(clean_review)
train
#creamos un df con solo la columna review_clean y sentiment_label
train_clean =train[['review_clean', 'sentiment_label']]
train_clean

,review_clean,sentiment_label
0,want great slice new york pizza come burke str...,0
1,way much breading shrimp also really loud next...,1
2,wa first time eaten peruvian food wa flavorful,0
3,ihop ha better food go south time get ham chee...,1
4,bowling giant jenga corn hole ping pong tried ...,1
...,...,...
4760,opinion frosted hawaiian tasted like blend ice...,1
4761,chicken sandwich double berger combo,1
4762,came try something new breakfast disappointed way,0
4763,daughter blackened fish sandwich wife shrimp d...,1


####2.2 Preprocesamos el conjunto de test

In [9]:
# Aplicar la función a la columna 'review' y añadirla al df para tener todo ahí
test['review_clean'] = test['review_text'].apply(clean_review)

#creamos un df con solo la columna review_clean y sentiment_label
test_clean =test[['review_clean', 'sentiment_label']]
test_clean

,review_clean,sentiment_label
0,wife really enjoy pasta fresh salad updated re...,1
1,pork cracklins boudin ball sweet potato beignet,0
2,salad appetizer main course came time,1
3,burger great fry good like onion ring better,1
4,always bomb place never disappoints best fast ...,0
...,...,...
1584,burger though wa average something could get n...,1
1585,good flavor personally prefer thicker noodle a...,1
1586,worth one time stop youre fan irish coffee,1
1587,wasnt fond root beer doughnut sweet dough seem...,1


In [10]:
#observamos el cambio en train
print('Review original: {}'.format(train['review_text'].values[0]))
print('Review procesada: {}'.format(train['review_clean'].values[0]))

Review original: When I want a great slice of New York Pizza I come here to Burke Street Pizza. The slices are cheap and they got the taste down to a perfection. Other recommendations if your not in the mood for pizza try the cheeseburger sub or the hot wings.
Review procesada: want great slice new york pizza come burke street pizza slice cheap got taste perfection recommendation mood pizza try cheeseburger sub hot wing


In [11]:
#observamos el cambio en test
print('Review original: {}'.format(test['review_text'].values[0]))
print('Review procesada: {}'.format(test['review_clean'].values[0]))

Review original: My wife and I really enjoy the pasta and fresh salads. Updated review:  today my wife and I had spaghetti and meatballs and lasagna. I love the salads and the soup Zuppa.
Review procesada: wife really enjoy pasta fresh salad updated review today wife spaghetti meatball lasagna love salad soup zuppa


###2.3. mezclamos nuestros datos con sus etiquetas

In [16]:
 # Mezclamos los datos con sus etiquetas
#Creamos los datos de train para nuestro modelo
x_train, y_train =shuffle(train_clean['review_clean'],train_clean['sentiment_label'])
print(y_train.tail(25))
#Creamos los datos de test
x_test,y_test =shuffle(test_clean['review_clean'],test_clean['sentiment_label'])


3570    0
4024    1
2517    1
3781    1
3240    0
1314    1
2202    1
490     0
4050    1
4045    1
1264    1
1657    0
1225    0
4044    1
99      1
1148    1
3701    1
962     1
3882    0
4454    1
2019    1
2616    0
4522    1
3619    0
60      0
Name: sentiment_label, dtype: int64


In [17]:
# guardamos los dfs en nuestro \content
train_clean.to_csv('train_clean.csv', encoding='utf-8', index=False)
test_clean.to_csv('test_clean.csv', encoding='utf-8', index=False)
x_train.to_csv('x_train.csv', encoding='utf-8', index=False)
x_test.to_csv('x_test.csv', encoding='utf-8', index=False)
y_train.to_csv('y_train.csv', encoding='utf-8', index=False)
y_test.to_csv('y_test.csv', encoding='utf-8', index=False)

In [18]:
#compruebo que los he guardado en mi disco de colab
!ls -lah   

total 1.3M
drwxr-xr-x 1 root root 4.0K Mar 10 12:03 .
drwxr-xr-x 1 root root 4.0K Mar 10 07:55 ..
drwxr-xr-x 4 root root 4.0K Mar  8 14:34 .config
drwx------ 5 root root 4.0K Mar 10 12:02 drive
drwxr-xr-x 1 root root 4.0K Mar  8 14:35 sample_data
-rw-r--r-- 1 root root 154K Mar 10 12:03 test_clean.csv
-rw-r--r-- 1 root root 472K Mar 10 12:03 train_clean.csv
-rw-r--r-- 1 root root 151K Mar 10 12:03 x_test.csv
-rw-r--r-- 1 root root 463K Mar 10 12:03 x_train.csv
-rw-r--r-- 1 root root 3.2K Mar 10 12:03 y_test.csv
-rw-r--r-- 1 root root 9.4K Mar 10 12:03 y_train.csv


In [19]:
# lo movemos a nuestro en nuestro google drive
!cp /content/train_clean.csv /content/drive/MyDrive/NLP/train_clean.csv
!cp /content/test_clean.csv /content/drive/MyDrive/NLP/test_clean.csv
!cp /content/x_train.csv /content/drive/MyDrive/NLP/x_train.csv
!cp /content/x_test.csv /content/drive/MyDrive/NLP/x_test.csv
!cp /content/y_train.csv /content/drive/MyDrive/NLP/y_train.csv
!cp /content/y_test.csv /content/drive/MyDrive/NLP/y_test.csv
